In [79]:
import pandas as pd 
import numpy as np
import plotly.express as px

In [80]:
riverflow_df = pd.read_csv("/Users/mugisha/Desktop/Drought_Pakistan/riverflow/mainInflowrivers.csv")
names = list(riverflow_df.columns)
names[1:]

['INDUS', 'KABUL', 'JEHLUM', 'CHENAB']

In [ ]:
fig = px.line(riverflow_df[riverflow_df[''] == country], 
    x = "year", y = "gdpPercap", title = country)